### If this notebook is helpful, please upvote [the original version](https://www.kaggle.com/code/leehann/inference-bert-for-uspatents)! (score: 0.8392)

### Version 2-8: 

```
# === add np.median ===

add_preds = []
for x in zip(*upd_predictions):
    add_preds.append(np.median(x, axis=0))
    
upd_predictions.append(add_preds)

# === add np.mean ===

[...]
```

### I am trying to improve my score: 0.8393

### Version 9: 

```
def _upd_score_between(data, thresholds, value):
    mask_th = data.between(*thresholds, inclusive='both')
    data[mask_th] = value


def upd_score(data, th_dict=None):
    [...]
    if th0:
        if isinstance(th0, float):
            th0 = (result.min(), th0)
        
        if isinstance(th0, tuple):
            _upd_score_between(result, th0, 0)
    
    if th25 and isinstance(th25, tuple):
        _upd_score_between(result, th25, 0.25)

    [...]
```

#### Calibrate scores (use thresholds)

```
thresholds_dict = {       # between(min_x, max_x, inclusive='both')
    '0': 0.02,            # (min_x, max_x) or X -> (data.min(), X)
    '.25': (0.24, 0.26),  # (min_x, max_x)
    '.50': (0.49, 0.51),  # (min_x, max_x)
    '.75': (0.74, 0.76),  # (min_x, max_x)
    '1': 0.98             # (min_x, max_x) or X -> (x, data.max())
}

submission['score'] = upd_score(submission['score'], thresholds_dict)

```

# 1. Import & Set & Def & Load

In [ ]:
import os
import pandas as pd
import numpy as np

from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig, AutoModel

from sklearn.preprocessing import MinMaxScaler

In [ ]:
class CFG_DEB_SIMPLE:
    input_path = '../input/us-patent-phrase-to-phrase-matching/'
    model_path = '../input/deberta-v3-large/deberta-v3-large'
    batch_size = 24
    num_workers = 2
    num_fold = 4
    max_input_length = 130

In [ ]:
class TestDataset(Dataset):
    def __init__(self, df, tokenizer, max_input_length):
        self.text = df['text'].values.astype(str)
        self.tokenizer = tokenizer
        self.max_input_length = max_input_length
        
    def __len__(self):
        return len(self.text)

    def __getitem__(self, item):
        inputs = self.text[item]
        
        inputs = self.tokenizer(inputs,
                    max_length=self.max_input_length,
                    padding='max_length',
                    truncation=True)
        
        return torch.as_tensor(inputs['input_ids'], dtype=torch.long), \
               torch.as_tensor(inputs['token_type_ids'], dtype=torch.long), \
               torch.as_tensor(inputs['attention_mask'], dtype=torch.long)
    
    
class Custom_Bert_Simple(nn.Module):
    def __init__(self, model_path):
        super().__init__()
        
        config = AutoConfig.from_pretrained(model_path)
        config.num_labels = 1
        self.base = AutoModelForSequenceClassification.from_config(config=config)
        dim = config.hidden_size
        self.dropout = nn.Dropout(p=0)
        self.cls = nn.Linear(dim,1)
        
    def forward(self, input_ids, attention_mask, token_type_ids, labels=None):
        base_output = self.base(input_ids=input_ids,
                                attention_mask=attention_mask,
                                token_type_ids=token_type_ids
        )

        return base_output[0]

In [ ]:
def valid_fn(valid_loader, model, device):
    model.eval()
    preds = []
    labels = []
    
    for step, batch in enumerate(valid_loader):
        input_ids, token_type_ids, attention_mask = [i.to(device) for i in batch]
    
        with torch.no_grad():
            y_preds = model(input_ids, attention_mask, token_type_ids)
        
        preds.append(y_preds.to('cpu').numpy())
    
    predictions = np.concatenate(preds)
    
    return predictions


min_max_scaler = MinMaxScaler()

def upd_outputs(data, is_trim=True, is_minmax=True, is_reshape=True):
    """\o/"""
    if is_trim == True:
        data = np.where(data <=0, 0, data)
        data = np.where(data >=1, 1, data)

    if is_minmax ==True:
        data = min_max_scaler.fit_transform(data)
    
    if is_reshape == True:
        data = data.reshape(-1)
        
    return data


def _upd_score_between(data, thresholds, value):
    """\o/"""
    mask_th = data.between(*thresholds, inclusive='both')
    data[mask_th] = value


def upd_score(data, th_dict=None):
    """\o/"""
    if isinstance(data, pd.Series):
        result = data.copy()
    else:
        return data

    if not th_dict:        
        th_dict = {
            '0': 0.02,
            '.25': (0.24, 0.26),
            '.50': (0.49, 0.51),
            '.75': (0.74, 0.76),
            '1': 0.98
        }

    if isinstance(th_dict, dict):    
        th0 = th_dict.get('0')
        th25 = th_dict.get('.25')
        th50 = th_dict.get('.50')
        th75 = th_dict.get('.75')
        th100 = th_dict.get('1')
    else:
        return data
    
    if th0:
        if isinstance(th0, float):
            th0 = (result.min(), th0)
        
        if isinstance(th0, tuple):
            _upd_score_between(result, th0, 0)
    
    if th25 and isinstance(th25, tuple):
        _upd_score_between(result, th25, 0.25)

    if th50 and isinstance(th50, tuple):
        _upd_score_between(result, th50, 0.50)
            
    if th75 and isinstance(th75, tuple):
        _upd_score_between(result, th75, 0.75)
            
    if th100:
        if isinstance(th100, float):
            th100 = (th100, result.max())
        
        if isinstance(th100, tuple):
            _upd_score_between(result, th100, 1)

    return result

In [ ]:
test_df = pd.read_csv(f"{CFG_DEB_SIMPLE.input_path}test.csv")
titles = pd.read_csv('../input/cpc-codes/titles.csv')
test_df = test_df.merge(titles, left_on='context', right_on='code')

cpc_texts = torch.load("../input/folds-dump-the-two-paths-fix/cpc_texts.pth")

test_df['context_text'] = test_df['context'].map(cpc_texts)
test_df['text'] = test_df['anchor'] + '[SEP]' + test_df['target'] + '[SEP]'  + test_df['context_text']
test_df['text'] = test_df['text'].apply(str.lower)

test_df.head()

# 2. Extract & Update Predictions

In [ ]:
tokenizer_deberta_v3 = AutoTokenizer.from_pretrained(CFG_DEB_SIMPLE.model_path)

In [ ]:
predictions = []

te_dataset = TestDataset(test_df, tokenizer_deberta_v3, CFG_DEB_SIMPLE.max_input_length)
te_dataloader = DataLoader(te_dataset,
                              batch_size=CFG_DEB_SIMPLE.batch_size, shuffle=False,
                              num_workers=CFG_DEB_SIMPLE.num_workers,
                              pin_memory=True, drop_last=False)

deberta_simple_path = "../input/us-patent-deberta-simple/microsoft_deberta-v3-large"

for fold in tqdm(range(CFG_DEB_SIMPLE.num_fold)):
    fold_path = f"{deberta_simple_path}_best{fold}.pth"
    
    model = Custom_Bert_Simple(CFG_DEB_SIMPLE.model_path)
    model.load_state_dict(torch.load(fold_path)['model'])
    model.to('cuda')
    
    prediction = valid_fn(te_dataloader, model, 'cuda')
    
    predictions.append(prediction)

In [ ]:
print("folds:", len(predictions))
print("rows: ", len(predictions[0]))
print("score:", predictions[0][0])

In [ ]:
n_predictions = 14

In [ ]:
# first fold
predictions[0][:n_predictions]

In [ ]:
# print(*upd_outputs(predictions[0], is_trim=False)[:n_predictions])
# print(*upd_outputs(predictions[0], is_minmax=False)[:n_predictions])

In [ ]:
# np.where(x<=0, 0, x) .. >> min_max.fit_transform(x) >> x.reshape(-1)
upd_predictions = [upd_outputs(x, is_trim=False) for x in predictions]

In [ ]:
print(*upd_predictions[0][:n_predictions])

# 3. Additional & Final Predictions

In [ ]:
origin_predictions = upd_predictions.copy()  # 5. Visualization

In [ ]:
# === add np.median ===
add_preds = []
for x in zip(*upd_predictions):
    add_preds.append(np.median(x, axis=0))
    
upd_predictions.append(add_preds)

In [ ]:
# === add np.mean ===
add_preds = []
for x in zip(*upd_predictions):
    add_preds.append(np.mean(x, axis=0))
    
upd_predictions.append(add_preds)

In [ ]:
final_predictions = np.mean(upd_predictions, axis=0)

In [ ]:
print(*final_predictions[:n_predictions])

# 4. Create & Calibrate Submissions

In [ ]:
submission = pd.DataFrame({
    'id': test_df['id'],
    'score': final_predictions,
})

submission.head(14)

In [ ]:
thresholds_dict = {
    '0': 0.02,
    '.25': (0.24, 0.26),
    '.50': (0.49, 0.51),
    '.75': (0.74, 0.76),
    '1': 0.98
}

submission['score'] = upd_score(submission['score'], thresholds_dict)

submission.head(14)

In [ ]:
submission.to_csv('submission.csv', index=False)

# 5. Visualization origin_predictions

In [ ]:
import seaborn as sns
cm = sns.light_palette('green', as_cmap=True)
props_param = "color:white; font-weight:bold; background-color:green;"
pd.set_option('display.precision', 10)

In [ ]:
df = pd.DataFrame(origin_predictions).T.head(15)
df = df.rename_axis(columns='folds', index='rows')

df.style.background_gradient(cmap=cm, axis=1)

In [ ]:
df.style.highlight_quantile(q_left=0.75, axis=1, color='green')

In [ ]:
df.assign(mean=lambda x: x.mean(axis=1)) \
    .style.highlight_max(axis=1, props=props_param)

In [ ]:
df.sub(df.mean(axis=1), axis=0) \
    .style.background_gradient(cmap=cm, axis=1)